In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define the input shape and number of classes
img_width, img_height = 260, 260  # Default image size for EfficientNetV2-S
num_classes = 2
train_dir = 'dataset_path/train'
val_dir = 'dataset_path/val'
epochs = 10

In [ ]:
# Add forms to allow users to choose different image sizes and epochs
img_width = input("Enter the image width (e.g., 224): ")  # Ask the user to input the image width
img_height = input("Enter the image height (e.g., 224): ")  # Ask the user to input the image height
img_width, img_height = int(img_width), int(img_height)  # Convert inputs to integers

epochs = int(input("Enter the number of epochs (e.g., 10): "))  # Ask the user to input the number of epochs

In [ ]:
# Create the data generators for training and validation sets
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_width, img_height), batch_size=32, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(img_width, img_height), batch_size=32, class_mode='binary')

In [ ]:
effnet_version = input("Enter the EfficientNetV2 version (e.g., EfficientNetV2S, EfficientNetV2M, etc.): ")

In [ ]:
# Load the specified EfficientNetV2 model version
if effnet_version in ["EfficientNetV2B0", "EfficientNetV2B1", "EfficientNetV2B2", "EfficientNetV2B3", "EfficientNetV2S", "EfficientNetV2M", "EfficientNetV2L"]:
    base_model = tf.keras.applications.__getattribute__(effnet_version)(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
else:
    print("Invalid EfficientNetV2 version. Please choose from the provided options.")
    exit(1)

In [ ]:
# Add custom top layers for your specific task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation="sigmoid")(x)

In [ ]:
# Freeze the pre-trained layers except the top layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model with your specific loss function and optimizer
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Train the model with your
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

In [ ]:
val_acc = history.history['val_accuracy']

In [ ]:
# Visulaize training results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Save the trained model
model.save("saved_model")

In [ ]:
# Test the model on a new image
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
# load new image
new_image_path = "path_to_new_image.jpg"
img = image.load_img(new_image_path, target_size=(img_width, img_height))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

In [ ]:
# Load the saved model
saved_model = tf.keras.models.load_model("saved_model")

In [ ]:
# Make predictions
predictions = saved_model.predict(img_array)
predicted_class = "Class1" if predictions[0][0] < 0.5 else "Class2"

print(f"Predicted Class: {predicted_class}")